# Algorithm

## Main Individual Method
1. Roll dice - done
2. Move to new position - done
3. Check what position type
4. If not-ownable, apply effect, move to step 8
5. If ownable, check if owned
    1. If owned, check if mortgaged
        1. if mortgaged, move to step 8
        2. if not mortgaged, determine amount and owner then pay
            1. If cannot pay, determine assets
                1. If owned utilites, mortgage
                2. Elif owned stations, mortgage
                3. Elif owned houses, sell
                4. Elif owned properties, mortgage 
                5. Else declare bankruptcy and quit game
6. If not owned, check cost of property relative to cash
    1. If cost is acceptable, buy property.
    2. If not acceptable, move to step 8.
7. If player has enough cash
    1. Check if own colour set
        1. if so, buy house on most expensive property first then any. Difference between number of houses on each property        cannot exceed 2
        2. if not, check if any trades possible
8. If rolled double, roll again (move to step 1), if not skip
    1. If 3rd double in a row, move to prison
    
## Determining if should buy

## Determing whether to build

## Prison

## Trading

 
## External to Player
1. determine how many players left
2. if > 1, end turn and go again
3. else, end game and crown winner

# Notes
- Use classes for players and properties

In [1]:
import random as rdm

In [2]:
class monopoly:
    
    def dice_throw(self):
        dice = [1, 2, 3, 4, 5, 6]
        dice_1 = rdm.choice(dice)
        dice_2 = rdm.choice(dice)
        total = dice_1 + dice_2
        return [total, dice_1 == dice_2]
    
    def _buy_property(self, player, space):
        if player.cash - space.value > 200:
            space.owned = True
            player.owned_properties.append(space)
            player.cash += -space.value
            space.owner = player.name
            print(player.name, 'bought', space.name, 'for', space.value)
            
    def _rent_utility(self, player, space, players):
        if space.owner != player.name:
            for i in players:
                if space in i.owned_properties:
                    dice = player.last_dice_throw[0]
                    player.cash += -(4 * dice)
                    i.cash += (4 * dice)
                    print('This utility is owned by', space.owner,', pay rent worth', (4 * dice))
        else:
            print('You own this property!')

    def _rent_station(self, player, space, players):
        if space.owner != player.name:
            for i in players:
                if space in i.owned_properties:
                    player.cash += -space.base_rent
                    i.cash += space.base_rent
                    print('This station is owned by', space.owner,', pay rent worth', space.base_rent)
        else:
            print('You own this property!')

    def _rent_street(self, player, space, players):
        if space.owner != player.name:
            for i in players:
                if space in i.owned_properties:
                    player.cash += -space.base_rent
                    i.cash += space.base_rent
                    print('This street is owned by', space.owner, ', pay rent worth', space.base_rent)
        else:
            print('You own this property!')

    def rent_buy(self, player, space, players):
        if space.space_type == 'effect':
            player.cash += space.cash_effect
            player.board_position += space.position_effect
            print('Pay effect')
        elif self.space_type == 'utility':
            if space.owned == True:
                player._rent_utility(player, space, players)
            else:
                player._buy_property(player, space)

        elif self.space_type == 'station':
            if space.owned == True:
                player._rent_station(player, space, players)
            else:
                player._buy_property(player, space)
        else:
            if space.owned == True:
                player._rent_street(player, space, players)
            else:
                player._buy_property(player, space)
        
class player(monopoly):
    
    def __init__(self, name, cash = 2000, owned_properties = [], board_position = 0, last_dice_throw = [], board_maximum = 40,
                 in_prison = False):
        self.name = name
        self.cash = cash
        self.owned_properties = []
        self.board_position = board_position
        self.last_dice_throw = last_dice_throw
        self.board_maximum = board_maximum
        self.in_prison = in_prison
    
    def move_piece(self, player, space):
        print('It is', player.name, '\'s turn.')
        player_pos = player.board_position
        player.last_dice_throw = player.dice_throw()
        player_pos += player.last_dice_throw[0]
        if player_pos >= player.board_maximum:
            player_pos += -player.board_maximum
            player.cash += 200
            print('Passed Go and got 200')
        player.board_position = player_pos
        print(player.name, 'rolled a', player.last_dice_throw[0])
        print(player.name, 'is at position:', space.name)

class street(monopoly):
    def __init__(self, name, space_type, value, colour, flat_cost, base_rent, rent_1house,
                 rent_2house, rent_3house, rent_4house, rent_hotel, owned = False, owner = None, houses = 0):
        self.name = name
        self.space_type = space_type
        self.value = value
        self.colour = colour
        self.flat_cost = flat_cost
        self.base_rent = base_rent
        self.rent_1house = rent_1house
        self.rent_2house = rent_2house
        self.rent_3house = rent_3house
        self.rent_4house = rent_4house
        self.rent_hotel = rent_hotel
        self.owned = owned
        self.owner = owner
        self.houses = houses

class station(monopoly):
    def __init__(self, name, space_type, value, base_rent, owned = False, owner = None):
        self.name = name
        self.space_type = space_type
        self.value = value
        self.base_rent = base_rent
        self.owned = owned
        self.owner = owner
        
class utility(monopoly):
    def __init__(self, name, space_type, value, base_rent = 50, owned = False, owner = None):
        self.name = name
        self.space_type = space_type
        self.value = value
        self.base_rent = base_rent
        self.owned = owned
        self.owner = owner
        
class effect(monopoly):
    def __init__(self, name, space_type, cash_effect, position_effect = 0):
        self.name = name
        self.space_type = space_type
        self.cash_effect = cash_effect
        self.position_effect = position_effect

In [6]:
old_kent_road = street('Old Kent Road', 'street', 60, 'brown', 50, 2, 10, 30, 90, 160, 250)
whitechapel_road = street('Whitechapel Road', 'street', 60, 'brown', 50, 4, 20, 60, 180, 320, 450)

angel_islington = street('Angel Islington', 'street', 100, 'aqua', 50, 6, 30, 90, 270, 400, 550)
euston_road = street('Euston Road', 'street', 100, 'aqua', 50, 6, 30, 90, 270, 400, 550)
pentonville_road = street('Pentonville Road', 'street', 120, 'aqua', 50, 8, 40, 100, 300, 450, 600)

pall_mall = street('Pall Mall', 'street', 140, 'pink', 100, 10, 50, 150, 450, 625, 750)
whitehall = street('Whitehall', 'street', 140, 'pink', 100, 10, 50, 150, 450, 625, 750)
northumberland_avenue = street('Northumberland Avenue', 'street', 160, 'pink', 100, 12, 60, 180, 500, 700, 900)

bow_street = street('Bow Street', 'street', 180, 'orange', 100, 14, 70, 200, 550, 750, 950)
marlborough_street = street('Marlborough Street', 'street', 180, 'orange', 100, 14, 70, 200, 550, 750, 950)
vine_street = street('Vine Street', 'street', 200, 'orange', 100, 16, 80, 220, 60, 800, 1000)

strand = street('The Strand', 'street', 220, 'red', 150, 18, 90, 250, 700, 875, 1050)
fleet_street = street('Fleet Street', 'street', 220, 'red', 150, 18, 90, 250, 700, 875, 1050)
trafalgar_square = street('Trafalgar Square', 'street', 240, 'red', 150, 20, 100, 300, 750, 925, 1100)

leicester_square = street('Leicester Square', 'street', 260, 'yellow', 150, 22, 110, 330, 800, 975, 1150)
coventry_street = street('Coventry Street', 'street', 260, 'yellow', 150, 22, 110, 330, 800, 975, 1150)
piccadilly = street('Piccadilly', 'street', 280, 'yellow', 150, 24, 120, 360, 850, 1025, 1200)

regent_street = street('Regent Street', 'street', 300, 'green', 200, 26, 130, 390, 900, 1100, 1275)
oxford_street = street('Oxford Street', 'street', 300, 'green', 200, 26, 130, 390, 900, 1100, 1275)
bond_street = street('Bond Street', 'street', 320, 'green', 200, 28, 150, 450, 1000, 1200, 1400)

park_lane = street('Park Lane', 'street', 350, 'blue', 200, 35, 175, 500, 1100, 1300, 1500)
mayfair = street('Mayfair', 'street', 400, 'blue', 200, 50, 200, 600, 1400, 1700, 2000)


kings_cross = station('King\'s Cross Station', 'station', 200, 25)
marylebone = station('Marylebone Station','station', 200, 25)
fenchurch_street = station('Fenchurch Street Station','station', 200, 25)
liverpool_street = station('Liverpool Street Station','station', 200, 25)

electric = utility('Electric Company', 'utility', 120)
water = utility('Water Works', 'utility', 120)

go = effect('Go', 'effect', 0)
tax100 = effect('Tax 100', 'effect', -100)
tax200 = effect('Income Tax', 'effect', -200)
go_to_prison = effect('Go to Prison', 'effect', 0, -1)
chance_1 = effect('Chance Card', 'effect', 0)
chance_2 = effect('Chance Card', 'effect', 0)
chance_3 = effect('Chance Card', 'effect', 0)
community_chest_1 = effect('Community Chest', 'effect', 0)
community_chest_2 = effect('Community Chest', 'effect',  0)
community_chest_3 = effect('Community Chest', 'effect', 0)
prison = effect('Prison', 'effect', 0)
free_parking = effect('Free Parking', 'effect', 0)

board = [go, old_kent_road, community_chest_1, whitechapel_road, tax200, kings_cross, angel_islington, chance_1, euston_road, 
         pentonville_road, prison, pall_mall, electric, whitehall, northumberland_avenue, marylebone, bow_street, 
         community_chest_2, marlborough_street, vine_street, free_parking, strand, chance_2, fleet_street, trafalgar_square, 
         fenchurch_street, leicester_square, coventry_street, water, piccadilly, go_to_prison, regent_street, oxford_street, 
         community_chest_3, bond_street, liverpool_street, chance_3, park_lane, tax100, mayfair]
brown = [old_kent_road, whitechapel_road]
aqua = [angel_islington, euston_road, pentonville_road]
pink = [pall_mall, whitehall, northumberland_avenue]
orange = [bow_street, marlborough_street, vine_street]
red = [strand, fleet_street, trafalgar_square]
yellow = [leicester_square, coventry_street, piccadilly]
green = [regent_street, oxford_street, bond_street]
blue = [park_lane, mayfair]
stations = [kings_cross, marylebone, fenchurch_street, liverpool_street]
utilities = [electric, water]

In [7]:
player_number = 2
players = []
turn = 1
turns_max = 2

c1 = 1
for i in range(player_number):
    s1 = 'Player ' + str(c1)
    players.append(player(s1))
    c1 += 1

while turn <= turns_max:   
    print('It is turn', turn)
    current_player = 1
    for i in players:        
        i.move_piece(i, board[i.board_position])
        
        board[i.board_position].rent_buy(i, board[i.board_position], players)
        
        current_player += 1
    turn += 1

for i in players:
    print(i.cash, len(i.owned_properties))

It is turn 1
It is Player 1 's turn.
Player 1 rolled a 4
Player 1 is at position: Go
Pay effect
It is Player 2 's turn.
Player 2 rolled a 9
Player 2 is at position: Go
Player 2 bought Pentonville Road for 120
It is turn 2
It is Player 1 's turn.
Player 1 rolled a 10
Player 1 is at position: Income Tax
Player 1 bought Northumberland Avenue for 160
It is Player 2 's turn.
Player 2 rolled a 4
Player 2 is at position: Pentonville Road
Player 2 bought Whitehall for 140
1640 1
1740 2


In [5]:
board[12].owner

In [6]:
board[1].

In [25]:
y = test_object(1)
y.test_trait
y.test_func_1()
y.test_trait

2

In [48]:
x = [[1, 2], [3, 4], [5,6]]

for i in x:
    if 3 in i:
        print (1)
    else:
        print(2)

2
1
2
